In [1]:
import os
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="openai/gpt-oss-120b", temperature=0.3)
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001884B33DD80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001884B4DFEB0>, model_name='openai/gpt-oss-120b', temperature=0.3, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
# from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

### tool creation

In [ ]:


@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [4]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [6]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Given 2 numbers a and b this tool returns their product
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


### Tools binding


In [8]:
llm_with_tools = llm.bind_tools([multiply])
llm_with_tools

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001884B33DD80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001884B4DFEB0>, model_name='openai/gpt-oss-120b', temperature=0.3, model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'Given 2 numbers a and b this tool returns their product', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

### Tools Calling

In [9]:
llm_with_tools.invoke("Who are you?")

AIMessage(content='I’m ChatGPT, a large language model created by OpenAI. I’ve been trained on a wide variety of text up through June\u202f2024, and I can help with answering questions, brainstorming ideas, explaining concepts, drafting or editing writing, solving problems, and much more. I don’t have personal experiences or consciousness—I generate responses based on patterns in the data I was trained on. If you have anything you’d like to discuss or need assistance with, just let me know!', additional_kwargs={'reasoning_content': 'The user asks "Who are you?" Likely wants a brief introduction. Should respond as ChatGPT, a large language model trained by OpenAI, with knowledge cutoff 2024-06, current date 2025-12-16. Provide description.'}, response_metadata={'token_usage': {'completion_tokens': 160, 'prompt_tokens': 133, 'total_tokens': 293, 'completion_time': 0.384401604, 'prompt_time': 0.006079639, 'queue_time': 0.049062351, 'total_time': 0.390481243, 'completion_tokens_details': {

In [11]:
llm_with_tools.invoke("Can you multiply 10 with 234").tool_calls

[{'name': 'multiply',
  'args': {'a': 10, 'b': 234},
  'id': 'fc_508d3943-7f5f-496c-8f54-be06ec65a077',
  'type': 'tool_call'}]

### tool execution

In [12]:
result = llm_with_tools.invoke("Can you multiply 10 with 234")

result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 10, 'b': 234},
 'id': 'fc_2e0ac072-f4ed-4064-9cc9-c761f661e9e7',
 'type': 'tool_call'}

In [13]:
result.tool_calls[0]['args']

{'a': 10, 'b': 234}

In [15]:
multiply.invoke(result.tool_calls[0])

ToolMessage(content='2340', name='multiply', tool_call_id='fc_2e0ac072-f4ed-4064-9cc9-c761f661e9e7')

In [23]:
query = HumanMessage("Can you multiply 10 with 234")
messages = [query]

messages

[HumanMessage(content='Can you multiply 10 with 234', additional_kwargs={}, response_metadata={})]

In [24]:
result = llm_with_tools.invoke(messages)

messages.append(result)

In [25]:
messages

[HumanMessage(content='Can you multiply 10 with 234', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'User wants to multiply 10 * 234. Use function multiply.', 'tool_calls': [{'id': 'fc_70abb588-cfa1-4959-a1fb-60b8a93ee3ec', 'function': {'arguments': '{"a":10,"b":234}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 137, 'total_tokens': 185, 'completion_time': 0.11059028, 'prompt_time': 0.00762176, 'queue_time': 0.05349609, 'total_time': 0.11821204, 'completion_tokens_details': {'reasoning_tokens': 15}}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_ff6aa7708c', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b2769-b296-7cb0-b786-56f187ebe9f3-0', tool_calls=[{'name': 'multiply', 'args': {'a': 10, 'b': 234}, 'id': 'fc_70abb588-cfa1-4959-a1fb-60b8a93ee3ec', 'type': 't

In [26]:
tool_messages = multiply.invoke(result.tool_calls[0])

In [27]:
messages.append(tool_messages)
messages

[HumanMessage(content='Can you multiply 10 with 234', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'User wants to multiply 10 * 234. Use function multiply.', 'tool_calls': [{'id': 'fc_70abb588-cfa1-4959-a1fb-60b8a93ee3ec', 'function': {'arguments': '{"a":10,"b":234}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 137, 'total_tokens': 185, 'completion_time': 0.11059028, 'prompt_time': 0.00762176, 'queue_time': 0.05349609, 'total_time': 0.11821204, 'completion_tokens_details': {'reasoning_tokens': 15}}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_ff6aa7708c', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b2769-b296-7cb0-b786-56f187ebe9f3-0', tool_calls=[{'name': 'multiply', 'args': {'a': 10, 'b': 234}, 'id': 'fc_70abb588-cfa1-4959-a1fb-60b8a93ee3ec', 'type': 't

In [30]:
final_result = llm_with_tools.invoke(messages)
final_result

AIMessage(content='The product of 10 and 234 is **2,340**.', additional_kwargs={'reasoning_content': 'We need to respond with the result. Provide answer.'}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 169, 'total_tokens': 205, 'completion_time': 0.107423238, 'prompt_time': 0.00654392, 'queue_time': 0.05364365, 'total_time': 0.113967158, 'completion_tokens_details': {'reasoning_tokens': 12}}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a28df4bce5', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b276a-2fb7-70c3-bf40-7fb895df1399-0', usage_metadata={'input_tokens': 169, 'output_tokens': 36, 'total_tokens': 205, 'output_token_details': {'reasoning': 12}})

In [33]:
final_result.content

'The product of 10 and 234 is **2,340**.'